In [163]:
# 先从简单的CNN开始写吧，嘶，主要是数据太少了orzzzzz
# 所以只能做分类了，完蛋，我开始写之前怎么就没想到这点呢
# 没事，3个类，33个数据一类，勉勉强强够了，大不了我自己再洗一年的数据

import pandas as pd
from utils import randomSplit, Accumulator
from torch import nn
import torch
import numpy as np
from sklearn.model_selection import KFold         # k折交叉验
from sklearn.preprocessing import StandardScaler  # 归一化


def rank2class(df):
    if df['LgRk'] <= 6:
        return 0
    elif 6 < df['LgRk'] <= 13:
        return 1
    else:
        return 2


In [164]:
class myDataset():
    def __init__(self):
        self.player = pd.read_csv('DLdata\outfielder_combined_from_raw_fillna.csv', header=0, encoding='utf-8')
        self.player = self.player.drop(['Player', 'Pos', 'Comp', 'Age', 'Born'], axis=1)
        scaler = StandardScaler()  # z score 归一化
        for col in self.player.columns:
            if col in ('Squad', "Season"): continue
            self.player[[col]] = scaler.fit_transform(self.player[[col]])


        self.league = pd.read_excel('DLdata/SquadPerformance2021.xlsx', sheet_name='Sheet1')
        self.league['tier'] = self.league.apply(rank2class, axis=1)
        self.league = self.league[['Squad', 'Season', 'tier']]  
        
        #self.size_stat = [] 结果是这个最大的数据集，是30

        # 生成连续的变量
        self.X = []  # in GNN: list of Data object; in CNN: list of 2-dim tensors; in MLs list of 1-dim tensors
        self.y = []
        for id in self.league.index:
            squad = self.league.at[id, 'Squad']
            season = self.league.at[id, 'Season']
            if season == 2021: continue  # 发现github的数据在2021年仅仅只有前几场比赛的，所以不得不舍弃了orzzz
            self.X.append(self.Squad2Player(squad, season)) 
            self.y.append(self.league.at[id, 'tier'])  # 作为监督的label
        #count = np.array(self.y)
        #print((count==0).sum()) 120
        #print((count==1).sum()) 140
        #print((count==2).sum()) 132


    def Squad2Player(self, squad, season):
        "输入squad的名字，输出球队球员的数据"
        season = str(season) + '-' + str(int(season) + 1)
        re = self.player[self.player['Squad'] == squad]
        #print(re.columns)
        re = re[re['Season'] == season].drop(['Squad', 'Season'], axis=1)
        re = torch.Tensor(re.values)  # shape [nplayers, features]
        # print(re.shape) [20, 151]
        #self.size_stat.append(re.shape[0])

        upBound = 30
        if re.shape[0] < upBound:
            a = (upBound - re.shape[0]) // 2
            b = upBound - re.shape[0] - a
            a = torch.zeros((a, re.shape[1]))
            b = torch.zeros((b, re.shape[1]))
            return torch.cat([a, re, b], dim=0).unsqueeze(0)
        elif re.shape[0] == upBound:
            return re.unsqueeze(0)
        else:
            return re[:upBound, :].unsqueeze(0)
        
    def KFolder(self, K=10, shuffle=True):
            KF = KFold(n_splits=K, shuffle=shuffle)  
            for train_index, test_index in KF.split(self.X):
                yield train_index, test_index
    
    def DataIter(self, batchsize, data_index):
        '''输入K折交叉验证给出来的index'''
        
        for i in range(0, data_index.shape[0], batchsize):
            upper = min(i + batchsize, data_index.shape[0])
            x_l = []
            y_l = []
            for j in range(i, upper):
                x_l.append(self.X[j].unsqueeze(0))
                y_l.append(self.y[j])
            yield torch.cat(x_l, dim=0), torch.LongTensor(y_l)

test = myDataset()

for train, valid in test.KFolder():
    for x, y in test.DataIter(7, train):
        print(x.shape)
        break
    break


torch.Size([7, 1, 30, 151])


In [165]:
def train(net, dataset, train_idx, valid_idx, batch_size, num_epochs, lr, device, verbose=0):
    # 初始化参数
    def init_weight(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weight)
    if verbose != 0:
        print(f'training on: {device}')
    net.to(device)

    optimizer = torch.optim.Adam([p for p in net.parameters() if p.requires_grad==True], lr=lr)
    loss = nn.CrossEntropyLoss(label_smoothing=0.1)

    #res = [] # 训练集梯度，训练集正确率，训练集总数
    recorder = Accumulator(3)
    validRecorder = Accumulator(3)

    for epoch in range(num_epochs):
         # 训练集梯度，训练集正确率，训练集总数
        net.train()
        for X, y in dataset.DataIter(batch_size, train_idx):
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            yhat = net(X)
            #print(yhat.shape) 
            #print(y.shape)
            l = loss(yhat, y)
            l.backward()
            optimizer.step()
            
            correct = (yhat.argmax(axis=1) == y).sum()
            acc = int(correct) / int(y.shape[0])

            recorder.add(float(l * X.shape[0]), correct, X.shape[0]) #说明lossfun里，梯度除以了bachsize

        # 正确率和平均损失
        net.eval()
        for X, y in dataset.DataIter(batch_size, valid_idx):
            X, y = X.to(device), y.to(device)
            yhat = net(X)
            #print(yhat)
            l = loss(yhat, y)
            correct = (yhat.argmax(axis=1) == y).sum()
            acc = int(correct) / int(y.shape[0])
            validRecorder.add(float(l * X.shape[0]), correct, X.shape[0]) #说明lossfun里，梯度除以了bachsize

        if verbose != 0:
            print(f'epoch = {epoch+1}')
            #print(recorder[1], recorder[2], recorder[0])
            print(f'训练集正确率：{recorder[1]/recorder[2]:.3f}，训练集平均loss {recorder[0]:.3f}.', end=' ')
            print(f'验证集正确率：{validRecorder[1]/validRecorder[2]:.3f}，验证集平均loss {validRecorder[0]:.3f}.')
        recorder.clear()
        validRecorder.clear()

    return recorder, validRecorder



In [166]:
# 定义网络

def get_net(isPCA):
    if isPCA: # 输入变成了[1, 30, 30]
        # 发现不PCA的已经很可以了，遂不写这部分了
        return torch.nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, padding=(0, 0), stride=3), # [1, 30, 30] -> [2, 12, 44]
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),   # [1, 12, 44] -> [1, 6, 22]
        nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=2),   # [1, 6, 22] -> [1, 2, 10]
        nn.Flatten(),
        nn.Linear(20, 12)
        , nn.ReLU(), nn.Dropout(p=0.5),
        nn.Linear(12, 3)
        )

    else:
        # 输入变成了[1, 30, 151]
        return torch.nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(3, 7), padding=(0, 0), stride=3), # [1, 30, 151] -> [1, 10, 49]
        nn.ReLU(), #nn.Dropout(p=0.5),
        nn.MaxPool2d(kernel_size=2, stride=2),   # [1, 10, 49] -> [1, 5, 24]
        nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(3, 6), stride=(1, 2)),   # [1, 5, 24] -> [1, 3, 10]
        nn.Flatten(), #nn.Dropout(p=0.5),
        nn.Linear(30, 12),
        nn.ReLU(), 
        nn.Dropout(p=0.5),
        nn.Linear(12, 3)
        )

x = torch.rand((40, 1, 30, 151))
model = get_net(0)
for layer in model.children():
    x = layer(x)
    print(f"{layer.__class__.__name__}: {x.shape}")


Conv2d: torch.Size([40, 1, 10, 49])
ReLU: torch.Size([40, 1, 10, 49])
MaxPool2d: torch.Size([40, 1, 5, 24])
Conv2d: torch.Size([40, 1, 3, 10])
Flatten: torch.Size([40, 30])
Linear: torch.Size([40, 12])
ReLU: torch.Size([40, 12])
Dropout: torch.Size([40, 12])
Linear: torch.Size([40, 3])


In [167]:
from tqdm import tqdm
ISPCA = False
dataset = myDataset()
# 训练超参数
lr = 0.01
num_epoch = 15
batch_size = 5
exp_stats_train = []
exp_stats_valid = []
EXPERIMENT = 5
K = 10
pybar = tqdm(total=EXPERIMENT*K, desc='k折交叉验证', unit='次')

for exp in range(EXPERIMENT):
    for train_idx, valid_idx in dataset.KFolder(K=K):
        # 正式训练
        pybar.update(1)
        net = get_net(False)
        train_rec, valid_rec = train(net, dataset, train_idx,
                        valid_idx,
                        batch_size, num_epoch, lr, device='cpu', verbose=0)
        # 取出验证集最后一个epoch的正确率
        #print(train_rec.read_backup(1))
        exp_stats_train.append(train_rec.read_backup(1)[-1]/train_rec.read_backup(2)[-1])  
        exp_stats_valid.append(valid_rec.read_backup(1)[-1]/valid_rec.read_backup(2)[-1])

exp_stats_train = np.array(exp_stats_train)
exp_stats_valid = np.array(exp_stats_valid)

print(f'训练集acc: mean = {exp_stats_train.mean():.3f}, std = {exp_stats_train.std():.3f}')
print(f'估计泛化误差：验证集acc: mean = {exp_stats_valid.mean():.3f}, std = {exp_stats_valid.std():.3f}')


k折交叉验证: 100%|██████████| 10/10 [00:45<00:00,  4.57s/次]


训练集acc: mean = 0.728, std = 0.094
验证集acc: mean = 0.844, std = 0.136
